In [1]:
from datetime import datetime, timedelta, date

In [2]:
import time
import pandas as pd
import numpy as np
from scipy import interpolate, stats
import matplotlib.pyplot as plt
import PyUber
from datetime import datetime, timedelta, date
import glob
import os

start =  time.time()

def main():
    spices = SQL_DataFrame(sql)
    spices.to_csv('spices.csv', index=False)

def convert_to_date(df, column1='MEAS_SET_DATA_COLLECT_DATE', column2='LOT_DATA_COLLECT_DATE', \
                    column3='CURRENT_MOVEIN_DATE', column4='END_DATE'):
    if column1 in df.columns:
        df[column1] = pd.to_datetime(df[column1])
    if column2 in df.columns:
        df[column2] = pd.to_datetime(df[column2])
    if column3 in df.columns:
        df[column3] = pd.to_datetime(df[column3])
    if column4 in df.columns:
        df[column4] = pd.to_datetime(df[column4])
    return df

def SQL_DataFrame(sql, source='D1D_PROD_XEUS'):
    conn = PyUber.connect(source)
    df = pd.read_sql(sql, conn)
    return df

def get90d(ss, datestr): 
    end = pd.Timestamp(datestr) + timedelta(days=1)
    start = end - timedelta(days=90)
    try:
        ss = ss[(ss['LOT_DATA_COLLECT_DATE'] >= start) & (ss['LOT_DATA_COLLECT_DATE'] < end)]
    except:
        ss = convert_to_date(ss)
        ss = ss[(ss['LOT_DATA_COLLECT_DATE'] >= start) & (ss['LOT_DATA_COLLECT_DATE'] < end)]
    return ss

sql="""
SELECT  DISTINCT 
          a1.entity AS entity
         ,a5.value AS chart_value
         ,To_Char(a0.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS lot_data_collect_date
         ,a3.measurement_set_name AS measurement_set_name
         ,To_Char(a3.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS meas_set_data_collect_date
         ,a2.monitor_type AS monitor_type
         ,a3.parameter_class AS parameter_class
         ,a2.monitor_set_name AS monitor_set_name
         ,a0.lotoperkey AS lotoperkey
         ,a5.incontrol_flag AS incontrol_flag
         ,a5.standard_flag AS chart_pt_standard_flag
         ,a10.centerline AS centerline
         ,a10.lo_control_lmt AS lo_control_lmt
         ,a10.up_control_lmt AS up_control_lmt
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a2.test_name AS test_name
         ,a3.parameter_header AS parameter_header
         ,a2.module AS module
FROM 
P_SPC_MEASUREMENT_SET a3
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a3.spcs_id
LEFT JOIN P_SPC_LOT a0 ON a0.spcs_id = a2.spcs_id
INNER JOIN P_SPC_ENTITY a1 ON a2.spcs_id = a1.spcs_id AND a1.entity_sequence=1
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
LEFT JOIN P_SPC_CHART_LIMIT a10 ON a10.chart_id = a5.chart_id AND a10.limit_id = a5.limit_id
WHERE
              (a1.entity LIKE 'LAT%' 
              OR a1.entity LIKE 'PAT%')
 AND      a5.value Is Not Null  
 AND      a3.data_collection_time >= TRUNC(SYSDATE) - 90 
 AND      a3.data_collection_time <= TRUNC(SYSDATE) - 0 
 AND      a2.monitor_type = 'TOOL MONITOR' 
 AND      a3.parameter_class = 'DEFECT_PARTICLE' 
 AND      a5.spc_chart_subset = 'PARTICLE_SIZE=TOTAL_ADDERS'
"""

sql2= '''SELECT 
          e.entity AS entity
         ,ea.attribute_value AS attribute_value
         ,e.ceid AS ceid
         ,ea.attribute_name AS attribute_name
         ,e.rev_module AS rev_module
FROM 
F_ENTITY e
LEFT JOIN F_ENTITYATTRIBUTE ea ON ea.entity = e.entity AND ea.history_deleted_flag='N'
WHERE
              (e.entity Like 'PAT%' 
              OR e.entity Like 'LAT%')
 AND      ea.attribute_name Like 'PM_Counter' 
ORDER BY
           1 Asc'''

sql3 = '''SELECT  DISTINCT 
          z0.entity AS entity
         ,z0.config_level_2 AS config_level_2
FROM 
F_UE_Detail z0
WHERE
              (z0.entity LIKE  'PAT%'
OR z0.entity LIKE 'LAT%'
) 
 AND      z0.rollup_date >= SYSDATE - 1 '''

# AND      

#if __name__ == "__main__": main()

In [3]:
ss = SQL_DataFrame(sql)

In [4]:
done1 = time.time()

In [5]:
print(done1-start)

299.53263902664185


tool_r = SQL_DataFrame(sql2)



sql3 = '''SELECT  DISTINCT 
          z0.entity AS entity
         ,z0.config_level_2 AS config_level_2
FROM 
F_UE_Detail z0
WHERE
              (z0.entity LIKE  'PAT%'
OR z0.entity LIKE 'LAT%'
) 
 AND      z0.rollup_date >= SYSDATE - 1 '''

tool_r = SQL_DataFrame(sql3)

tool_r['ATTRIBUTE_VALUE'] = tool_r['ATTRIBUTE_VALUE'].astype(int)
tools = pd.pivot_table(tool_r, values = 'ATTRIBUTE_VALUE', index = ['REV_MODULE', 'CEID' ,'ENTITY'], columns = 'ATTRIBUTE_NAME')
tools = tools.reset_index()
tools.set_index(['ENTITY'], inplace = True)
#tools[tools['ENTITY'] == 'LATXX']['REV_MODULE'].values[0]

camp = os.getcwd()

In [6]:
ss.head()

,ENTITY,CHART_VALUE,LOT_DATA_COLLECT_DATE,MEASUREMENT_SET_NAME,MEAS_SET_DATA_COLLECT_DATE,MONITOR_TYPE,PARAMETER_CLASS,MONITOR_SET_NAME,LOTOPERKEY,INCONTROL_FLAG,CHART_PT_STANDARD_FLAG,CENTERLINE,LO_CONTROL_LMT,UP_CONTROL_LMT,CHART_TYPE,SPC_CHART_SUBSET,TEST_NAME,PARAMETER_HEADER,MODULE
0,PAT414,27.750000,2022-01-31 13:19:33,LAT.DSA_PST.74.DER,2022-01-31 13:19:33,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.577330e+09,Y,Y,50.0,0.0,100.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSAL,PARTICLE_SIZE,LAT
1,PAT422,207.000000,2022-03-26 22:22:10,LAT.DSA_PST.74.DER,2022-03-26 22:22:10,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.703076e+09,Y,Y,50.0,0.0,100.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSAL,PARTICLE_SIZE,LAT
2,PAT466,6.000000,2022-03-28 03:12:36,LAT.DSA_PST.74.DER,2022-03-28 03:12:36,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.707492e+09,Y,Y,50.0,0.0,100.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSAL,PARTICLE_SIZE,LAT
3,PAT410_PM2,746.666667,2022-01-22 10:35:15,LAT.DSA_PST.74.DER,2022-01-22 10:35:15,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.551330e+09,Y,Y,10.0,0.0,32.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSOS,PARTICLE_SIZE,LAT
4,PAT466_PM2,43.666667,2022-01-21 19:39:36,LAT.DSA_PST.74.DER,2022-01-21 19:39:36,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.557166e+09,Y,Y,10.0,0.0,32.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSOS,PARTICLE_SIZE,LAT


# DO NOT RUN THIS
import glob
sta=pd.DataFrame()
#csvs = glob.glob('training_data/LAT*.csv')
#for fnode in csvs:
if 1==1:
    dstr = fnode.split('.')[-2]
    ss = SQL_DataFrame(sql)
    entities = ss.ENTITY.unique()
    for entity in entities:
        #entity=entities[0]
        try:
            fname = tools.loc[entity]['REV_MODULE']+'.TA.'+entity+'.'+dstr
        except:
            fname = 'NONE.TA.'+entity+'.'+dstr
        sst = ss[ss['ENTITY']==entity]
        st = sst[sst['SPC_CHART_SUBSET'] == 'PARTICLE_SIZE=TOTAL_ADDERS']

        st=st[['LOT_DATA_COLLECT_DATE', 'CHART_VALUE']]
        st=st.rename(columns={'LOT_DATA_COLLECT_DATE': 't', 'CHART_VALUE': fname})
        st.index = pd.to_datetime(st.t)
        st.drop(['t'], axis=1, inplace = True)
        st=st.sort_index()
        st=st.resample('3D').mean()

        st=st.interpolate(method='linear')
        st=np.log(st+1)
        st = st[len(st)-20:]
        if len(st)<20: continue
        
        if 1==2: #don't plot
            fig = plt.figure()
            ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
            #ax = plt.subplot(111)
            ax.plot(st, 'o')
            ax.set_xlabel('date')
            ax.set_ylabel('TA')
            ax.set_xticks(ax.get_xticks()[::3])
            #plt.title('title')
            ax.legend([entity+': '+dstr])
            fig.savefig('figs/'+fname)
            plt.close()
            st = st.reset_index().T.drop('t')

        st = st.reset_index().T.drop('t')
        if sta.empty: sta=st
        else: sta = sta.append(st)
            


    entities = ss.ENTITY.unique()
    sta=pd.DataFrame()
    for entity in entities:
        #entity=entities[0]
        try:
            fname = tools.loc[entity]['REV_MODULE']+'.TA.'+entity+'.'+dstr
        except:
            fname = 'NONE.TA.'+entity+'.'+dstr
        sst = ss[ss['ENTITY']==entity]
        st = sst[sst['SPC_CHART_SUBSET'] == 'PARTICLE_SIZE=TOTAL_ADDERS']

        st=st[['LOT_DATA_COLLECT_DATE', 'CHART_VALUE']]
        st=st.rename(columns={'LOT_DATA_COLLECT_DATE': 't', 'CHART_VALUE': fname})
        st.index = pd.to_datetime(st.t)
        st.drop(['t'], axis=1, inplace = True)
        st=st.sort_index()
        st=st.resample('3D').mean()

        st=st.interpolate(method='linear')
        st=np.log(st+1)
        st = st[len(st)-20:]
        if len(st)<20: continue
        
        if 1==2: #don't plot
            fig = plt.figure()
            ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
            #ax = plt.subplot(111)
            ax.plot(st, 'o')
            ax.set_xlabel('date')
            ax.set_ylabel('TA')
            ax.set_xticks(ax.get_xticks()[::3])
            #plt.title('title')
            ax.legend([entity+': '+dstr])
            fig.savefig('figs/'+fname)
            plt.close()
            st = st.reset_index().T.drop('t')

        st = st.reset_index().T.drop('t')
        if sta.empty: sta=st
        else: sta = sta.append(st)

In [ ]:
import time

start = time.time()
sta=pd.DataFrame()
st=[0]

ssb = ss

# snapdates = ['2021-04-14', '2021-03-31', '2021-03-17', '2021-01-21', '2020-12-23', '2020-11-24', '2020-10-18', \
#             '2020-09-20', '2020-08-23', '2020-07-21', '2020-06-23', '2020-05-25', '2020-04-22', '2020-03-25', \
#             '2020-02-25']

#training_dates=pd.read_csv('training_data/training_dates.csv')
#snapdates = training_dates[training_dates['Module'] == 'LAT']['Date']
snapdates = [str(datetime.now())[:10]]

# csvs = glob.glob('training_data/LAT*.csv')
# csvs.reverse()

#Get module/tool information
# tool_r = SQL_DataFrame(sql2)
# tool_r['ATTRIBUTE_VALUE'] = tool_r['ATTRIBUTE_VALUE'].astype(int)
# tools = pd.pivot_table(tool_r, values = 'ATTRIBUTE_VALUE', index = ['REV_MODULE', 'CEID' ,'ENTITY'], columns = 'ATTRIBUTE_NAME')
# tools = tools.reset_index()
# tools.set_index(['ENTITY'], inplace = True)

tools = SQL_DataFrame(sql3).rename(columns={'CONFIG_LEVEL_2': 'REV_MODULE'}).set_index('ENTITY')

#Calculate module baselines
taa = ssb[ssb['SPC_CHART_SUBSET'] == 'PARTICLE_SIZE=TOTAL_ADDERS']
taa = pd.merge(taa, tools, on='ENTITY')
taa=taa[['REV_MODULE', 'CHART_VALUE']]
taa=taa.rename(columns={'REV_MODULE': 'module', 'CHART_VALUE': 'TA'})
taac = taa[taa.groupby('module').TA.transform(lambda x: stats.zscore(x)<0.5)]
baselines = taac.groupby('module').mean()

#existing_dates = []

for dstr in snapdates:
    ss = get90d(ssb, dstr)
    entities = ss.ENTITY.unique()
    
#     taa = ss[ss['SPC_CHART_SUBSET'] == 'PARTICLE_SIZE=TOTAL_ADDERS']
#     taa = pd.merge(taa, tools, on='ENTITY')
#     taa=taa[['REV_MODULE', 'CHART_VALUE']]
#     taa=taa.rename(columns={'REV_MODULE': 'module', 'CHART_VALUE': 'TA'})
#     taac = taa[taa.groupby('module').TA.transform(lambda x: stats.zscore(x)<0.5)]
#     baselines = taac.groupby('module').mean()
    
    for entity in entities:
        #if len(st)==20 : break
            
        #entity=entities[0]
        try:
            #tools.loc[entity].values[0]
            fname = tools.loc[entity]['REV_MODULE']+'.TA.'+entity+'.'+dstr 
            if len(fname)==2: fname = fname.values[0]
            baseline = baselines.loc[tools.loc[entity]['REV_MODULE']].values
        except:
            fname = 'NONE.TA.'+entity+'.'+dstr
            baseline = baselines['TA'].mean()
        sst = ss[ss['ENTITY']==entity]
        st = sst[sst['SPC_CHART_SUBSET'] == 'PARTICLE_SIZE=TOTAL_ADDERS']

        st=st[['LOT_DATA_COLLECT_DATE', 'CHART_VALUE']]
        st=st.rename(columns={'LOT_DATA_COLLECT_DATE': 't', 'CHART_VALUE': fname})
        st.index = pd.to_datetime(st.t)
        st.drop(['t'], axis=1, inplace = True)
        if len(st)<20: continue # need enough data to interpolate properly
        st=st.sort_index()
        st=st.resample('3D').mean()
        try:
            st=st.interpolate(method='spline', order=2)
            st[st<0] = 0
        except:
            continue
        #st=np.log(st+1)
        st = st[len(st)-20:]
        if len(st)<20: continue #sometimes data doesn't extend back 60 days
        #print(fname+' len: ', len(st))
        
        st = st/baseline
        
        want_figs = False
        if ((len(st)==20) & (st.mean() < np.inf)[0] & want_figs) : #do plot
            # PLOT the smoothed data
            fig = plt.figure()
            ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
            #ax = plt.subplot(111)
            x = np.linspace(-len(st)+1,0,len(st)).reshape(-1,1)*3
            ax.plot(x,st.values, 'o-')
            ax.legend([entity+': '+dstr])
            
            # PLOT spline
            try: 
                xn = np.linspace(x[0], x[-1], 100)
                non_fliers = st.values.astype(float)<4
                yp = st[non_fliers]
                xp = x[non_fliers]
                y_BSpline = interpolate.UnivariateSpline(xp,yp,s=20.)
                yn = y_BSpline(xn)
                ax.plot(xn, yn, '-')
            except:
                pass
            
            ax.set_xlabel('day')
            ax.set_ylabel('TA')
            ax.set_xticks([-60,-30,0])
            plt.ylim([-0.1, 4])
            
            fig.savefig('figs/'+fname+'.png')
            plt.close()
            #break

        #if len(st)==20 : break
        st = st.reset_index().T.drop('t')
        if sta.empty: sta=st
        else: sta = sta.append(st)
        #break
    
    #break

print('seconds: ', time.time()-start)

In [ ]:
sta

In [ ]:
sta.head()

In [ ]:
import keras
from keras.models import load_model
model = load_model('ss_trend_5LYRLSTM_20210426.h5')
sta = sta.rename_axis('key')
sta = sta.reset_index()
y=sta.drop(['key'], axis=1).values
pred = model.predict(y.reshape(-1,20,1))
keys = pd.DataFrame(sta['key'])
dkeys = keys
dkeys['pred'] = pred
dkeys = keys[pred>0.55]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

dstr = str(datetime.now())[:10]

for key in dkeys.key:
    #plt.plot(sta[sta['key'] == key].drop(['key'], axis =1, inplace=True))
    print(key)
    st = sta[sta['key'] == key].drop(['key'], axis=1).T
    #ta = np.exp(a.T.values.astype(float))-1
#     st = a.T
#     fig = plt.figure()
#     ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
#     ax.semilogy(np.arange(-59,0,3), ta,'o')
#     ax.legend([key.replace('.', '  ')])
#     ax.set_xlabel('days')
#     ax.set_ylabel('TA')
#     ax.set_xticks(ax.get_xticks()[::4])
#     fig.savefig('output/'+key+'.png')
#     plt.close()
    
   
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    #ax = plt.subplot(111)
    x = np.linspace(-len(st)+1,0,len(st)).reshape(-1,1)*3
    ax.plot(x, st, 'o-')
    
    entity = key.split('.')[2]
    predstr = str( round(dkeys[dkeys['key']==key]['pred'].values[0], 2))
    ax.legend([entity+': '+dstr + ' ' + predstr])

    # PLOT spline
    try: 
        xn = np.linspace(x[0], x[-1], 100)
        non_fliers = st.values.astype(float)<4
        yp = st[non_fliers]
        xp = x[non_fliers]
        y_BSpline = interpolate.UnivariateSpline(xp,yp,s=20.)
        yn = y_BSpline(xn)
        ax.plot(xn, yn, '-')
    except:
        pass

    ax.set_xlabel('day')
    ax.set_ylabel('TA')
    ax.set_xticks([-60,-30,0])
    plt.ylim([-0.1, 4])

    fig.savefig('//orshfs.intel.com/ORAnalysis$/1274_MAODATA/Etch/etch_flagger/LAT.SS.TRENDING/'+key+'.png')
    plt.close()

In [8]:
ss.head()

,ENTITY,CHART_VALUE,LOT_DATA_COLLECT_DATE,MEASUREMENT_SET_NAME,MEAS_SET_DATA_COLLECT_DATE,MONITOR_TYPE,PARAMETER_CLASS,MONITOR_SET_NAME,LOTOPERKEY,INCONTROL_FLAG,CHART_PT_STANDARD_FLAG,CENTERLINE,LO_CONTROL_LMT,UP_CONTROL_LMT,CHART_TYPE,SPC_CHART_SUBSET,TEST_NAME,PARAMETER_HEADER,MODULE
0,PAT414,27.750000,2022-01-31 13:19:33,LAT.DSA_PST.74.DER,2022-01-31 13:19:33,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.577330e+09,Y,Y,50.0,0.0,100.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSAL,PARTICLE_SIZE,LAT
1,PAT422,207.000000,2022-03-26 22:22:10,LAT.DSA_PST.74.DER,2022-03-26 22:22:10,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.703076e+09,Y,Y,50.0,0.0,100.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSAL,PARTICLE_SIZE,LAT
2,PAT466,6.000000,2022-03-28 03:12:36,LAT.DSA_PST.74.DER,2022-03-28 03:12:36,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.707492e+09,Y,Y,50.0,0.0,100.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSAL,PARTICLE_SIZE,LAT
3,PAT410_PM2,746.666667,2022-01-22 10:35:15,LAT.DSA_PST.74.DER,2022-01-22 10:35:15,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.551330e+09,Y,Y,10.0,0.0,32.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSOS,PARTICLE_SIZE,LAT
4,PAT466_PM2,43.666667,2022-01-21 19:39:36,LAT.DSA_PST.74.DER,2022-01-21 19:39:36,TOOL MONITOR,DEFECT_PARTICLE,LAT.DSA_PST.74.MON,3.557166e+09,Y,Y,10.0,0.0,32.0,X-BAR,PARTICLE_SIZE=TOTAL_ADDERS,4LATNTSSOS,PARTICLE_SIZE,LAT


In [ ]:
baselines

In [ ]:
y

In [ ]:
pred = model.predict(y.reshape(-1,20,1))

In [ ]:
np.sum(pred)

In [ ]:
np.sum((pred>0.5)*1)

In [ ]:
len(sta['key'])

In [ ]:
len(pred>0.5)

In [ ]:
keys = pd.DataFrame(sta['key'])
dkeys = keys
dkeys['pred'] = pred
dkeys = keys[pred>0.55]

In [ ]:
dtools = dkeys.key.apply(lambda x: x.split('.')[2])

In [ ]:
#dkeys.sort_values(by='key')
len(dkeys)

In [ ]:
dkeys

In [ ]:
print(keys.key[0])
print(len(dkeys.key))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

a = sta[sta['key'] == 'DE-LAT-TNC.TA.PAT464_PM4.2021-03-07'].drop(['key'], axis=1)

In [ ]:
a

In [ ]:
plt.plot(np.arange(20), a.T.values,'o')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

dstr = str(datetime.now())[:10]

for key in dkeys.key:
    #plt.plot(sta[sta['key'] == key].drop(['key'], axis =1, inplace=True))
    print(key)
    st = sta[sta['key'] == key].drop(['key'], axis=1).T
    #ta = np.exp(a.T.values.astype(float))-1
#     st = a.T
#     fig = plt.figure()
#     ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
#     ax.semilogy(np.arange(-59,0,3), ta,'o')
#     ax.legend([key.replace('.', '  ')])
#     ax.set_xlabel('days')
#     ax.set_ylabel('TA')
#     ax.set_xticks(ax.get_xticks()[::4])
#     fig.savefig('output/'+key+'.png')
#     plt.close()
    
   
    fig = plt.figure()
    ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
    #ax = plt.subplot(111)
    x = np.linspace(-len(st)+1,0,len(st)).reshape(-1,1)*3
    ax.plot(x, st, 'o-')
    
    entity = key.split('.')[2]
    predstr = str( round(dkeys[dkeys['key']==key]['pred'].values[0], 2))
    ax.legend([entity+': '+dstr + ' ' + predstr])

    # PLOT spline
    try: 
        xn = np.linspace(x[0], x[-1], 100)
        non_fliers = st.values.astype(float)<4
        yp = st[non_fliers]
        xp = x[non_fliers]
        y_BSpline = interpolate.UnivariateSpline(xp,yp,s=20.)
        yn = y_BSpline(xn)
        ax.plot(xn, yn, '-')
    except:
        pass

    ax.set_xlabel('day')
    ax.set_ylabel('TA')
    ax.set_xticks([-60,-30,0])
    plt.ylim([-0.1, 4])

    fig.savefig('output/'+key+'.png')
    plt.close()

In [ ]:
#key = dkeys.key[0]
round(dkeys[dkeys['key']==key]['pred'].values[0],2)

In [ ]:
a = sta[sta['key'] == 'DE-LAT.TA.LAT418_PM2.2021-03-07'].drop(['key'], axis=1)
ta = np.exp(a.T.values.astype(float))-1
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.semilogy(np.arange(20), ta,'o')


In [ ]:
len(np.arange(-59,0,3))

In [ ]:
a.T.values.astype(float)

In [ ]:
dkeys.sort_values(by='key').key.values

In [ ]:
from email.message import EmailMessage
from email.utils import make_msgid
import mimetypes

msg = EmailMessage()

# generic email headers
msg['Subject'] = 'Hello there'
msg['From'] = 'ABCD <abcd@xyz.com>'
msg['To'] = 'PQRS <pqrs@xyz.com>'

# set the plain text body
msg.set_content('This is a plain text body.')

# now create a Content-ID for the image
image_cid = make_msgid(domain='xyz.com')
# if `domain` argument isn't provided, it will 
# use your computer's name

# set an alternative html body
msg.add_alternative("""\
<html>
    <body>
        <p>This is an HTML body.<br>
           It also has an image.
        </p>
        <img src="cid:{image_cid}">
    </body>
</html>
""".format(image_cid=image_cid[1:-1]), subtype='html')
# image_cid looks like <long.random.number@xyz.com>
# to use it as the img src, we don't need `<` or `>`
# so we use [1:-1] to strip them off


# now open the image and attach it to the email
with open('path/to/image.jpg', 'rb') as img:

    # know the Content-Type of the image
    maintype, subtype = mimetypes.guess_type(img.name)[0].split('/')

    # attach it
    msg.get_payload()[1].add_related(img.read(), 
                                         maintype=maintype, 
                                         subtype=subtype, 
                                         cid=image_cid)



In [ ]:
a = sta[sta['key'] == key].drop(['key'], axis=1)

In [ ]:
st = a.T

In [ ]:
st

In [ ]:
tools = tool_r
tools.rename(columns={'CONFIG_LEVEL_2': 'REV_MODULE'}).set_index('ENTITY')
#tools.set_index('ENTITY')

In [ ]:
sta[sta['key'] == key]

In [ ]:
key

In [ ]:
st

In [ ]:
        st = sst[sst['SPC_CHART_SUBSET'] == 'PARTICLE_SIZE=TOTAL_ADDERS']
        st=st[['LOT_DATA_COLLECT_DATE', 'CHART_VALUE']]
        #st=st.rename(columns={'LOT_DATA_COLLECT_DATE': 't', 'CHART_VALUE': fname})
        #st.index = pd.to_datetime(st.t)

In [ ]:
st.rename(columns={'LOT_DATA_COLLECT_DATE': 't', 'CHART_VALUE': fname})

In [ ]:
fname

In [ ]:
tools.loc['PAT410_PM2']

In [ ]:
tools.loc['PAT410_PM2'].iloc[0]+'.TA.'+entity+'.'+dstr ##tools.loc[entity].values[0]

In [ ]:
len(tools.loc['PAT410_PM2'])

In [ ]:
fname = tools.loc['PAT410_PM2']['REV_MODULE']+'.TA.'+entity+'.'+dstr 

In [ ]:
fname

In [ ]:
if len(fname)>1: fname = fname.values[0]

fname

In [ ]:
tools

In [ ]:
fname

In [ ]:
len(tools.loc['PAT410_PM2']['REV_MODULE']+'.TA.'+entity+'.'+dstr )

In [ ]:
import lib

In [ ]:
2+2

In [ ]:
resample_20d(st)

In [ ]:
import torch

In [ ]:
a=torch.tensor([1,2,3])

In [ ]:
a.dtype

In [ ]:
a.tolist()